In [1]:
import pandas as pd
import geopandas as gpd
import py7zr, tempfile

### Extract Metropole
This notebook takes a list of municipality identifiers that belong to the Lyon Metropole and generates a unique polygon file that covers this area.

In [2]:
# Manage inputs and outputs
iris_path = "../../resources/external/iris/CONTOURS-IRIS_2-1__SHP__FRA_2022-01-01.7z"
metropole_path = "../../resources/spatial/metropole.csv"
output_path = "../../results/metropole.gpkg"

if "snakemake" in locals():
    iris_path = snakemake.input["iris"]
    metropole_path = snakemake.input["metropole"]
    output_path = snakemake.output[0]

In [3]:
# Load municipality identifiers
df_metropole = pd.read_csv(metropole_path, sep = ";", dtype = {
    "municipality_id": str
})

In [6]:
# Load IRIS data
with tempfile.TemporaryDirectory(delete = True) as directory:
    with py7zr.SevenZipFile(iris_path) as archive:
        archive.extractall(path = directory)

    source_path = "/".join([
        directory, 
        "CONTOURS-IRIS_2-1__SHP__FRA_2022-01-01",
        "CONTOURS-IRIS",
        "1_DONNEES_LIVRAISON_2022-06-00180",
        "CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2022",
        "CONTOURS-IRIS.shp"
    ])

    df_iris = gpd.read_file(source_path)

In [7]:
# Cleanup IRIS data
df_iris = df_iris[["INSEE_COM", "geometry"]].rename(columns = { 
    "INSEE_COM": "municipality_id"
})

df_iris = df_iris[df_iris["municipality_id"].isin(
    df_metropole["municipality_id"]
)]

In [8]:
# Save a single polygon covering the Metropole
df_iris.dissolve().to_file(output_path)